In [2]:
from google.colab import drive
import pandas as pd

drive.mount('/gdrive',force_remount=True)

snp400 = pd.read_csv('/gdrive/MyDrive/BA870 Group Project /new dataset/Stock_data_snp400.csv')
snp500 = pd.read_csv('/gdrive/MyDrive/BA870 Group Project /new dataset/Stock_data_snp500.csv')
snp600 = pd.read_csv('/gdrive/MyDrive/BA870 Group Project /new dataset/Stock_data_snp600.csv')

Mounted at /gdrive


## EDA and Cleaning

In [3]:
snp500.head(130)

,gvkey,iid,datadate,tic,cshtrm,prccm,trt1m,cmth,cshom,cyear,gind,gsector
0,1078,01,2015-01-31,ABT,116068599.0,44.76,-0.0444,1.0,1.505791e+09,2015.0,351010,35
1,1078,01,2015-02-28,ABT,112821349.0,47.37,5.8311,2.0,1.505791e+09,2015.0,351010,35
2,1078,01,2015-03-31,ABT,113223820.0,46.33,-2.1955,3.0,1.508978e+09,2015.0,351010,35
3,1078,01,2015-04-30,ABT,97933528.0,46.42,0.7123,4.0,1.508978e+09,2015.0,351010,35
4,1078,01,2015-05-31,ABT,88975776.0,48.60,4.6963,5.0,1.488757e+09,2015.0,351010,35
...,...,...,...,...,...,...,...,...,...,...,...,...
125,1161,01,2015-06-30,AMD,388806275.0,2.40,5.2632,6.0,7.780600e+08,2015.0,453010,45
126,1161,01,2015-07-31,AMD,404968509.0,1.93,-19.5833,7.0,7.791780e+08,2015.0,453010,45
127,1161,01,2015-08-31,AMD,252130743.0,1.81,-6.2176,8.0,7.791780e+08,2015.0,453010,45
128,1161,01,2015-09-30,AMD,187846051.0,1.72,-4.9724,9.0,7.791780e+08,2015.0,453010,45


In [4]:
snp500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9294 entries, 0 to 9293
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gvkey     9294 non-null   int64  
 1   iid       9294 non-null   object 
 2   datadate  9294 non-null   object 
 3   tic       9294 non-null   object 
 4   cshtrm    9167 non-null   float64
 5   prccm     9169 non-null   float64
 6   trt1m     9160 non-null   float64
 7   cmth      9189 non-null   float64
 8   cshom     9189 non-null   float64
 9   cyear     9189 non-null   float64
 10  gind      9294 non-null   int64  
 11  gsector   9294 non-null   int64  
dtypes: float64(6), int64(3), object(3)
memory usage: 871.4+ KB


In [5]:
# Renaming columns
col_map = {
    # identifiers
    'tic'     : 'ticker',            # Ticker Symbol
    'gind'    : 'gics_industry',     # GICS Industry code
    'gsector' : 'gics_sector',       # GICS Sector code
    # technical metrics
    'trt1m'   : 'monthly_return',   # % total return for the month
    'cshtrm'  : 'month_trading_volume', # Trading Volume – Monthly
    'prccm'   : 'price_close_monthly',    # Closing Price – Monthly
    'cshom'   : 'shares_outstanding_monthly',  # Shares Outstanding – Monthly
    # calendar helpers
    'cmth'    : 'calendar_month',
    'cyear'   : 'calendar_year'
}

snp500 = snp500.rename(columns=col_map)

In [6]:
snp400 = snp400.rename(columns=col_map)
snp600 = snp600.rename(columns=col_map)

In [7]:
snp500.head()

,gvkey,iid,datadate,ticker,month_trading_volume,price_close_monthly,monthly_return,calendar_month,shares_outstanding_monthly,calendar_year,gics_industry,gics_sector
0,1078,01,2015-01-31,ABT,116068599.0,44.76,-0.0444,1.0,1.505791e+09,2015.0,351010,35
1,1078,01,2015-02-28,ABT,112821349.0,47.37,5.8311,2.0,1.505791e+09,2015.0,351010,35
2,1078,01,2015-03-31,ABT,113223820.0,46.33,-2.1955,3.0,1.508978e+09,2015.0,351010,35
3,1078,01,2015-04-30,ABT,97933528.0,46.42,0.7123,4.0,1.508978e+09,2015.0,351010,35
4,1078,01,2015-05-31,ABT,88975776.0,48.60,4.6963,5.0,1.488757e+09,2015.0,351010,35


In [8]:
# How many securities per company?
snp500.groupby('gvkey')['iid'].nunique().value_counts().head()

# See which IIDs occur
snp500['iid'].value_counts().head()

# Are there multiple IIDs for the same month?
dup = snp500.duplicated(subset=['gvkey', 'calendar_month', 'calendar_year'], keep=False)
snp500[dup].sort_values(['gvkey', 'calendar_year', 'calendar_month']).head()

,gvkey,iid,datadate,ticker,month_trading_volume,price_close_monthly,monthly_return,calendar_month,shares_outstanding_monthly,calendar_year,gics_industry,gics_sector
841,2176,01,2015-01-31,BRK.A,7162.0,215865.00,-4.4845,1.0,8.320000e+05,2015.0,402010,40
961,2176,02,2015-01-31,BRK.B,85901437.0,143.91,-4.1558,1.0,1.215933e+09,2015.0,402010,40
842,2176,01,2015-02-28,BRK.A,4520.0,221180.00,2.4622,2.0,8.320000e+05,2015.0,402010,40
962,2176,02,2015-02-28,BRK.B,64099222.0,147.41,2.4321,2.0,1.215933e+09,2015.0,402010,40
843,2176,01,2015-03-31,BRK.A,7962.0,217500.00,-1.6638,3.0,8.250000e+05,2015.0,402010,40


In [9]:
# Cleaning function (modify if required)

def clean_compustat_security(df: pd.DataFrame) -> pd.DataFrame:
    """Return a copy of a Compustat security‑level DataFrame
    with standardised dtypes."""
    df = df.copy()

    # datadate → pandas datetime
    df['datadate'] = pd.to_datetime(df['datadate'], errors='coerce')

    # tidy up char columns (strip spaces, force upper‑case tickers)
    for col in ['ticker', 'iid']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip().str.upper()

    # calendar fields → nullable ints (retain NaNs cleanly)
    if 'calendar_month' in df.columns:
        df['calendar_month'] = (
            pd.to_numeric(df['calendar_month'], errors='coerce')
              .round()
              .astype('Int8')          # 1‑12 fits in Int8
        )

    if 'calendar_year' in df.columns:
        df['calendar_year'] = (
            pd.to_numeric(df['calendar_year'], errors='coerce')
              .round()
              .astype('Int16')         # covers 1900‑2100 easily
        )

    return df

for name in ['snp400', 'snp500', 'snp600']:
    globals()[name] = clean_compustat_security(globals()[name])

print(snp500.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9294 entries, 0 to 9293
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   gvkey                       9294 non-null   int64         
 1   iid                         9294 non-null   object        
 2   datadate                    9294 non-null   datetime64[ns]
 3   ticker                      9294 non-null   object        
 4   month_trading_volume        9167 non-null   float64       
 5   price_close_monthly         9169 non-null   float64       
 6   monthly_return              9160 non-null   float64       
 7   calendar_month              9189 non-null   Int8          
 8   shares_outstanding_monthly  9189 non-null   float64       
 9   calendar_year               9189 non-null   Int16         
 10  gics_industry               9294 non-null   int64         
 11  gics_sector                 9294 non-null   int64       

In [10]:
import numpy as np

def patch_missing_security_panel(df: pd.DataFrame) -> pd.DataFrame:
    """
    Tidy a Compustat monthly security panel:
      • fix calendar_month / calendar_year
      • fill price, volume, shares
      • recompute missing returns from price
    """
    df = df.sort_values(['gvkey', 'iid', 'datadate']).copy()

    # calendar helpers
    df['calendar_month'] = df['datadate'].dt.month.astype('Int8')
    df['calendar_year']  = df['datadate'].dt.year.astype('Int16')

    # price
    df['price_close_monthly'] = (
        df.groupby(['gvkey', 'iid'])['price_close_monthly']
          .transform(lambda s: s.ffill().bfill())
    )

    # volume
    df['month_trading_volume'] = (
        df.groupby(['gvkey', 'iid'])['month_trading_volume']
          .ffill()                                   # forward only
    )

    # shares
    df['shares_outstanding_monthly'] = (
        df.groupby(['gvkey', 'iid'])['shares_outstanding_monthly']
          .ffill()
    )

    # return
    # first recompute % change based on price
    price_pct = (
        df.groupby(['gvkey', 'iid'])['price_close_monthly']
          .pct_change()*100          # Compustat returns are in %
    )

    # where original return is NA, use the recomputed one
    df['monthly_return'] = df['monthly_return'].mask(df['monthly_return'].isna(), price_pct)

    # final sanity: if any core numeric still missing, drop the rows
    core_cols = [
        'price_close_monthly', 'month_trading_volume',
        'shares_outstanding_monthly', 'monthly_return'
    ]
    df = df.dropna(subset=core_cols)

    return df

# --------------------------------------------------------
# apply to S&P 400 first
# --------------------------------------------------------
snp400_clean = patch_missing_security_panel(snp400)

print("Missing counts after cleaning:")
print(snp400_clean[[
    'price_close_monthly','month_trading_volume',
    'shares_outstanding_monthly','monthly_return'
]].isna().sum())


Missing counts after cleaning:
price_close_monthly           0
month_trading_volume          0
shares_outstanding_monthly    0
monthly_return                0
dtype: int64


<ipython-input-10-65d45b666384>:38: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change()*100          # Compustat returns are in %


In [11]:
snp500_clean = patch_missing_security_panel(snp500)

print("Missing counts after cleaning:")
print(snp500_clean[[
    'price_close_monthly','month_trading_volume',
    'shares_outstanding_monthly','monthly_return'
]].isna().sum())


Missing counts after cleaning:
price_close_monthly           0
month_trading_volume          0
shares_outstanding_monthly    0
monthly_return                0
dtype: int64


<ipython-input-10-65d45b666384>:38: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change()*100          # Compustat returns are in %


In [12]:
snp600_clean = patch_missing_security_panel(snp600)

print("Missing counts after cleaning:")
print(snp600_clean[[
    'price_close_monthly','month_trading_volume',
    'shares_outstanding_monthly','monthly_return'
]].isna().sum())


Missing counts after cleaning:
price_close_monthly           0
month_trading_volume          0
shares_outstanding_monthly    0
monthly_return                0
dtype: int64


<ipython-input-10-65d45b666384>:38: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .pct_change()*100          # Compustat returns are in %


In [13]:
# ╔══════════════════════════════════════════════════════════════╗
# ║  Build gvkey → gics_sector lookup from the clean technical   ║
# ║  tables, then add the column to every fundamental DataFrame. ║
# ╚══════════════════════════════════════════════════════════════╝
import pandas as pd

def build_gics_sector_map(*tech_dfs, prefer='mode'):
    """
    Return a Series mapping gvkey ➜ gics_sector.
    Accepts either 'gics_sector' or the original 'gsector' column name.
    """
    tech = pd.concat(tech_dfs, ignore_index=True)

    # tolerate either column name
    sector_col = next((c for c in ['gics_sector', 'gsector'] if c in tech.columns), None)
    if sector_col is None:
        raise KeyError("No sector column ('gics_sector' or 'gsector') in technical data.")

    tech = tech.dropna(subset=[sector_col])

    if prefer == 'mode':          # most frequent sector per gvkey
        gics_map = tech.groupby('gvkey')[sector_col].agg(lambda s: s.mode().min())
    elif prefer == 'first':       # take the first appearance
        gics_map = (tech.sort_values(['gvkey', 'datadate'])
                         .drop_duplicates('gvkey')
                         .set_index('gvkey')[sector_col])
    else:
        raise ValueError("prefer must be 'mode' or 'first'")

    return gics_map.astype('Int64')

# build once, AFTER technical cleaning:
_gics_map = build_gics_sector_map(snp400_clean, snp500_clean, snp600_clean)

def add_gics_sector(fund_df: pd.DataFrame, gics_map=_gics_map) -> pd.DataFrame:
    """Attach a gics_sector column to a fundamental DataFrame."""
    fund_df = fund_df.copy()
    fund_df['gics_sector'] = fund_df['gvkey'].map(gics_map)
    missing = fund_df['gics_sector'].isna().sum()
    if missing:
        print(f"⚠️  {missing:,} fundamentals rows still lack a sector code "
              "(will fall back to global medians).")
    return fund_df


### Import and Clean


In [14]:
snp400_fund = pd.read_csv('/gdrive/MyDrive/BA870 Group Project /new dataset/snp400.csv')
snp500_fund = pd.read_csv('/gdrive/MyDrive/BA870 Group Project /new dataset/snp500.csv')
snp600_fund = pd.read_csv('/gdrive/MyDrive/BA870 Group Project /new dataset/snp600.csv')

In [15]:
snp400_fund.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5803 entries, 0 to 5802
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gvkey     5803 non-null   int64  
 1   datadate  5803 non-null   object 
 2   fyearq    5803 non-null   int64  
 3   fqtr      5803 non-null   int64  
 4   indfmt    5803 non-null   object 
 5   consol    5803 non-null   object 
 6   popsrc    5803 non-null   object 
 7   datafmt   5803 non-null   object 
 8   tic       5803 non-null   object 
 9   curcdq    5803 non-null   object 
 10  datacqtr  5800 non-null   object 
 11  datafqtr  5803 non-null   object 
 12  actq      4323 non-null   float64
 13  atq       5664 non-null   float64
 14  ceqq      5663 non-null   float64
 15  dlcq      4964 non-null   float64
 16  dlttq     5626 non-null   float64
 17  dpq       5127 non-null   float64
 18  dvpq      5771 non-null   float64
 19  glaq      334 non-null    float64
 20  glpq      456 non-null    floa

In [16]:
# Mapping fundamentals data to rename columns

_fund_col_map = {
    # identifiers / housekeeping
    'tic'     : 'ticker',
    'fyearq'  : 'fiscal_year',
    'fqtr'    : 'fiscal_quarter',

    # balance‑sheet & leverage
    'actq'    : 'current_assets',
    'atq'     : 'total_assets',
    'ceqq'    : 'common_equity_total',
    'uceqq'   : 'common_equity_utility',
    'dlcq'    : 'current_debt',
    'dlttq'   : 'long_term_debt',
    'lctq'    : 'current_liabilities',
    'ltq'     : 'total_liabilities',

    # income statement
    'niitq'   : 'net_interest_income_tax_equiv',
    'niq'     : 'net_income',
    'piq'     : 'pretax_income',
    'revtq'   : 'total_revenue',
    'txtq'    : 'total_income_taxes',
    'xintq'   : 'interest_expense_total',

    # cash‑flow & capex
    'oancfy'  : 'net_cash_flow_operating_activities',
    'capxy'   : 'capital_expenditures',

    # dividends & gains / losses
    'dvpq'    : 'preferred_dividends',
    'dvpspq'  : 'dividends_per_share_quarter',
    'glaq'    : 'gain_loss_after_tax',
    'glpq'    : 'gain_loss_pretax',

    # depreciation, price
    'dpq'     : 'depreciation_amortization',
    'prclq'   : 'price_low_quarter',
}

# Applying mapping

def rename_fundamental_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a copy of a Compustat *quarterly* fundamental DataFrame
    with intuitive column names and proper datadate dtype.
    """
    df = df.copy()
    df = df.rename(columns=_fund_col_map)

    # ensure 'datadate' is datetime for downstream joins / resampling
    if 'datadate' in df.columns:
        df['datadate'] = pd.to_datetime(df['datadate'], errors='coerce')

    return df

snp400_fund = rename_fundamental_columns(snp400_fund)
snp500_fund = rename_fundamental_columns(snp500_fund)
snp600_fund = rename_fundamental_columns(snp600_fund)

In [17]:
# checking for redundancy
for name in ['snp400_fund', 'snp500_fund', 'snp600_fund']:
    df = globals()[name]
    keep = (df['indfmt'] == 'INDL') & (df['datafmt'] == 'STD') & \
           (df['consol'] == 'C') & (df['popsrc'] == 'D')
    globals()[name] = df.loc[keep].copy()

In [18]:
# Removing redundant columns

_META_COLS = ['indfmt', 'consol', 'popsrc', 'datafmt']

def drop_meta_cols(df):
    """Return a copy without the constant metadata columns."""
    return df.drop(columns=_META_COLS, errors='ignore')

# apply to all three fundamental panels
snp400_fund = drop_meta_cols(snp400_fund)
snp500_fund = drop_meta_cols(snp500_fund)
snp600_fund = drop_meta_cols(snp600_fund)

In [19]:
snp400_fund.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5803 entries, 0 to 5802
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   gvkey                               5803 non-null   int64         
 1   datadate                            5803 non-null   datetime64[ns]
 2   fiscal_year                         5803 non-null   int64         
 3   fiscal_quarter                      5803 non-null   int64         
 4   ticker                              5803 non-null   object        
 5   curcdq                              5803 non-null   object        
 6   datacqtr                            5800 non-null   object        
 7   datafqtr                            5803 non-null   object        
 8   current_assets                      4323 non-null   float64       
 9   total_assets                        5664 non-null   float64       
 10  common_equity_total          

In [20]:
# after drop_meta_cols, BEFORE align_with_technical or imputation:
snp400_fund = add_gics_sector(snp400_fund)
snp500_fund = add_gics_sector(snp500_fund)
snp600_fund = add_gics_sector(snp600_fund)

⚠️  2,939 fundamentals rows still lack a sector code (will fall back to global medians).
⚠️  2,919 fundamentals rows still lack a sector code (will fall back to global medians).
⚠️  2,859 fundamentals rows still lack a sector code (will fall back to global medians).


In [21]:
# Displaying % values missing in each column
na_pct = snp400_fund.isna().mean().sort_values(ascending=False)
display(na_pct.to_frame('% missing').style.format({'% missing': '{:.1%}'}))

,% missing
common_equity_utility,97.7%
net_interest_income_tax_equiv,97.7%
gain_loss_after_tax,94.2%
gain_loss_pretax,92.1%
gics_sector,50.6%
current_assets,25.5%
current_liabilities,25.5%
current_debt,14.5%
interest_expense_total,12.3%
depreciation_amortization,11.6%


In [22]:
# Dropping sparse columns (> 90% values misssing)
DROP_SPARSE = [
    'gain_loss_after_tax', 'gain_loss_pretax',
    'net_interest_income_tax_equiv', 'common_equity_utility'
]

DROP_METADATA = [
    'curcdq',      # constant currency code (USD) in all rows
    'datacqtr',    # duplicates datadate + fiscal quarter
    'datafqtr',    # idem
    'costat'       # status flag, but constant 'A' (active) here
]

snp400_fund = snp400_fund.drop(columns=DROP_SPARSE + DROP_METADATA)

In [23]:
# helper: keep only gvkeys that exist in the technical panel

def align_with_technical(fund_df, tech_df):
    """
    Return a copy of `fund_df` containing only gvkeys that also
    appear in `tech_df`.
    """
    keep = fund_df['gvkey'].isin(tech_df['gvkey'])
    dropped = (~keep).sum()
    print(f"Dropping {dropped:,} rows ({dropped/len(fund_df):.1%}) "
          f"that have no technical data.")
    return fund_df.loc[keep].copy()

# apply to each universe
snp400_fund = align_with_technical(snp400_fund, snp400_clean)
snp500_fund = align_with_technical(snp500_fund, snp500_clean)
snp600_fund = align_with_technical(snp600_fund, snp600_clean)

Dropping 2,939 rows (50.6%) that have no technical data.
Dropping 2,919 rows (49.8%) that have no technical data.
Dropping 2,859 rows (50.4%) that have no technical data.


In [24]:
# robust two‑sided fill + sector & global medians

def impute_fundamentals(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(['gvkey', 'datadate']).copy()

    #  numeric columns, skipping identifiers
    num_cols = df.select_dtypes('number').columns.difference(
        ['gvkey', 'fiscal_year', 'fiscal_quarter']
    )

    # firm‑level two‑sided fill
    grp = df.groupby('gvkey')[num_cols]
    df[num_cols] = grp.transform('ffill')
    df[num_cols] = grp.transform('bfill')

    # sector‑by‑quarter median fill
    sector_medians = (
        df.groupby(['gics_sector', 'datadate'])[num_cols]
          .transform('median')
    )
    df[num_cols] = df[num_cols].fillna(sector_medians)

    # global (cross‑sectional) median fill per quarter
    global_medians = df.groupby('datadate')[num_cols].transform('median')
    df[num_cols] = df[num_cols].fillna(global_medians)

    return df

In [25]:
# Applying safe division to ensure we dont divide with 0 or N/A
def safe_div(a, b):
    return np.where((b == 0) | b.isna(), np.nan, a / b)

def add_key_ratios(df):
    df = df.copy()
    df['current_ratio']   = safe_div(df['current_assets'], df['current_liabilities'])
    df['debt_to_equity']  = safe_div(df['current_debt'] + df['long_term_debt'],
                                     df['common_equity_total'])
    df['profit_margin']   = safe_div(df['net_income'], df['total_revenue'])
    df['roa']             = safe_div(df['net_income'], df['total_assets'])
    df['roe']             = safe_div(df['net_income'], df['common_equity_total'])
    df['capex_to_assets'] = safe_div(df['capital_expenditures'], df['total_assets'])
    return df

In [26]:
# re‑run the full prep wrapper

def prepare_fund_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.drop(columns=DROP_SPARSE + DROP_METADATA, errors='ignore')
    df = impute_fundamentals(df)   # <- updated helper
    df = add_key_ratios(df)
    return df

# cleaned, aligned fundamentals (after dropping unmatched gvkeys)
snp400_fund = prepare_fund_df(snp400_fund)
snp500_fund = prepare_fund_df(snp500_fund)
snp600_fund = prepare_fund_df(snp600_fund)

# optional sanity‑check
print("Any NA left?", pd.concat([snp400_fund, snp500_fund, snp600_fund])
                       .select_dtypes('number').isna().sum().any())

Any NA left? True


In [27]:
def show_remaining_nas(df, top=15):
    na = df.select_dtypes('number').isna().sum()
    na = na[na > 0].sort_values(ascending=False)
    print(f"{len(na)} numeric columns still contain NaNs")
    display(na.to_frame('count').head(top))

# after concatenation
all_fund = pd.concat([snp400_fund, snp500_fund, snp600_fund])
show_remaining_nas(all_fund)


4 numeric columns still contain NaNs


,count
profit_margin,30
current_debt,10
long_term_debt,10
debt_to_equity,10


In [28]:
def final_nudge(df):
    df = df.copy()

    # 1) debt fields – replace NaN with 0, then recompute debt_to_equity
    debt_cols = ['current_debt', 'long_term_debt']
    df[debt_cols] = df[debt_cols].fillna(0)

    # 2) profit_margin – safe division
    zero_rev = (df['total_revenue'] == 0) | df['total_revenue'].isna()
    df.loc[zero_rev & df['net_income'].isna(), 'net_income'] = 0        # if both NaN
    df.loc[zero_rev, 'profit_margin'] = 0

    # 3) recompute debt_to_equity with the patched debt
    df['debt_to_equity'] = (
        (df['current_debt'] + df['long_term_debt'])
        / df['common_equity_total']
        .replace({0: np.nan})        # avoid divide‑by‑zero
    )

    # any residual NaN in debt_to_equity → fill with cross‑sectional median
    median_d2e = df['debt_to_equity'].median(skipna=True)
    df['debt_to_equity'] = df['debt_to_equity'].fillna(median_d2e)

    return df

# apply to each fundamentals set
snp400_fund = final_nudge(snp400_fund)
snp500_fund = final_nudge(snp500_fund)
snp600_fund = final_nudge(snp600_fund)

# double‑check
all_fund = pd.concat([snp400_fund, snp500_fund, snp600_fund])
print("Numeric NaNs remaining:",
      all_fund.select_dtypes('number').isna().sum().sum())


Numeric NaNs remaining: 0


In [29]:
all_fund.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8612 entries, 204 to 5630
Data columns (total 30 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   gvkey                               8612 non-null   int64         
 1   datadate                            8612 non-null   datetime64[ns]
 2   fiscal_year                         8612 non-null   int64         
 3   fiscal_quarter                      8612 non-null   int64         
 4   ticker                              8612 non-null   object        
 5   current_assets                      8612 non-null   float64       
 6   total_assets                        8612 non-null   float64       
 7   common_equity_total                 8612 non-null   float64       
 8   current_debt                        8612 non-null   float64       
 9   long_term_debt                      8612 non-null   float64       
 10  depreciation_amortization  

In [30]:
all_tech = pd.concat([snp400_clean, snp500_clean, snp600_clean])
all_tech.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26105 entries, 0 to 8526
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   gvkey                       26105 non-null  int64         
 1   iid                         26105 non-null  object        
 2   datadate                    26105 non-null  datetime64[ns]
 3   ticker                      26105 non-null  object        
 4   month_trading_volume        26105 non-null  float64       
 5   price_close_monthly         26105 non-null  float64       
 6   monthly_return              26105 non-null  float64       
 7   calendar_month              26105 non-null  Int8          
 8   shares_outstanding_monthly  26105 non-null  float64       
 9   calendar_year               26105 non-null  Int16         
 10  gics_industry               26105 non-null  int64         
 11  gics_sector                 26105 non-null  int64         
d

In [31]:
  all_fund.groupby('gvkey')['ticker'].value_counts()

,,count
gvkey,ticker,
1078,ABT,40
1161,AMD,40
1300,HON,40
1447,AXP,40
1602,AMGN,40
...,...,...
186278,AL,40
189459,CG,40
190455,GWRE,40


Some things to note:

- We dropped 50% of the data in the fundamentals data as these companies did not have most/any technical data. This can be looked into later if we find a solution. GPT said this is due to companies joining/leaving the index fund during the time period of our analysis.

- We have a total of 223 companies left now after all the cleaning, dropping, etc.

- Once the datasets are merged we will have 26K+ rows in total.




## Merging Datasets

In [32]:
# Step 1: Rename fund datadate to distinguish after merge
all_fund = all_fund.rename(columns={'datadate': 'fund_date'})
all_tech = all_tech.rename(columns={'datadate': 'tech_date'})

# Step 2: Merge on ticker
combined = all_tech.merge(all_fund, on='ticker')

# Step 3: Keep only valid matches (fund_date <= tech_date)
combined = combined[combined['fund_date'] <= combined['tech_date']]

# Step 4: For each tech row, keep only the most recent fund row
combined['date_diff'] = (combined['tech_date'] - combined['fund_date']).dt.days

# Get the row with smallest positive difference per tech_date + ticker
combined = combined.sort_values(['ticker', 'tech_date', 'date_diff'])
combined = combined.groupby(['ticker', 'tech_date'], as_index=False).first()

# Final cleanup
combined = combined.drop(columns='date_diff')

Some rows were dropped because no matching quarterly (fundamental) data existed for that month and ticker.
This happens when the monthly date is earlier than the company's first available quarterly report.

In [33]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24634 entries, 0 to 24633
Data columns (total 41 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ticker                              24634 non-null  object        
 1   tech_date                           24634 non-null  datetime64[ns]
 2   gvkey_x                             24634 non-null  int64         
 3   iid                                 24634 non-null  object        
 4   month_trading_volume                24634 non-null  float64       
 5   price_close_monthly                 24634 non-null  float64       
 6   monthly_return                      24634 non-null  float64       
 7   calendar_month                      24634 non-null  Int8          
 8   shares_outstanding_monthly          24634 non-null  float64       
 9   calendar_year                       24634 non-null  Int16         
 10  gics_industry         

In [34]:
# Dropping redundant columns

combined = combined.drop(columns=[
    'gvkey_y', 'gics_sector_y',  # duplicates of gvkey_x, gics_sector_x
    'iid'                        # Compustat security ID—not used in modeling
])

## Model

#### Computing Sharpe Ratio

In [35]:
sharpe = (
  combined.groupby('ticker')['monthly_return']
          .agg(['mean','std'])
          .assign(sharpe=lambda df: df['mean']/df['std']*np.sqrt(12))
)
combined = combined.merge(
    sharpe['sharpe'].rename('sharpe_ratio'),
    on='ticker', how='left'
)

In [36]:
combined.head()

,ticker,tech_date,gvkey_x,month_trading_volume,price_close_monthly,monthly_return,calendar_month,shares_outstanding_monthly,calendar_year,gics_industry,...,net_cash_flow_operating_activities,dividends_per_share_quarter,price_low_quarter,current_ratio,debt_to_equity,profit_margin,roa,roe,capex_to_assets,sharpe_ratio
0,AAPL,2015-03-31,1690,1.136136e+09,124.430,-3.1372,3,5.824748e+09,2015,452020,...,52803.0,0.47,104.63,1.156005,0.340069,0.233908,0.051950,0.105181,0.021386,0.938336
1,AAPL,2015-04-30,1690,9.906444e+08,125.150,0.5786,4,5.761030e+09,2015,452020,...,52803.0,0.47,104.63,1.156005,0.340069,0.233908,0.051950,0.105181,0.021386,0.938336
2,AAPL,2015-05-31,1690,9.493128e+08,130.280,4.5146,5,5.761030e+09,2015,452020,...,52803.0,0.47,104.63,1.156005,0.340069,0.233908,0.051950,0.105181,0.021386,0.938336
3,AAPL,2015-06-30,1690,8.766773e+08,125.425,-3.7266,6,5.761030e+09,2015,452020,...,67791.0,0.52,123.10,1.086819,0.432999,0.215240,0.039088,0.084956,0.027930,0.938336
4,AAPL,2015-07-31,1690,1.048373e+09,121.300,-3.2888,7,5.702722e+09,2015,452020,...,67791.0,0.52,123.10,1.086819,0.432999,0.215240,0.039088,0.084956,0.027930,0.938336


In [37]:
ticker_sharpe = pd.DataFrame(combined.groupby('ticker')['sharpe_ratio'].unique())
ticker_sharpe[ticker_sharpe['sharpe_ratio'] > 1]

,sharpe_ratio
ticker,
AVGO,[1.2284958992281545]
CAVA,[1.3397031733509648]
COST,[1.118807121439476]
CSWI,[1.135967344000589]
EME,[1.0161850441237945]
FIX,[1.0964550513077798]
HLI,[1.148990005714805]
JXN,[1.1112157473031405]
KNSL,[1.2315620067839912]


We have calculated forward returns with a horizon of 12 months in order to calculate alpha. This calculation has led to NaN values in the rows belonging to the tail (last 12 months). We have dropped these rows, leading to ~ 22K rows remaining.

In [38]:
# ── 1) New Signals ────────────────────────────────────────────────
# 1a) Multi‑horizon momentum
for h in (1, 3, 6, 12):
    combined[f'mom_{h}m'] = (
        combined.groupby('gvkey_x')['price_close_monthly']
                .pct_change(periods=h)
    )

# 1b) Rolling return volatility
for w in (3, 6, 12):
    combined[f'vol_{w}m'] = (
        combined.groupby('gvkey_x')['monthly_return']
                .rolling(window=w)
                .std()
                .reset_index(level=0, drop=True)
    )

# 1c) Valuation ratios (reuse your safe_div from notebook)
def safe_div(a, b):
    return np.where((b == 0) | b.isna(), np.nan, a / b)

combined['pe_ratio'] = safe_div(
    combined['price_close_monthly'],
    combined['net_income'] / combined['shares_outstanding_monthly']
)
combined['pb_ratio'] = safe_div(
    combined['price_close_monthly'],
    combined['common_equity_total'] / combined['shares_outstanding_monthly']
)
combined['div_yield'] = safe_div(
    combined['dividends_per_share_quarter'],
    combined['price_close_monthly']
)

In [39]:
# for each stock (gvkey), compute:
for w in (3, 6, 12):
    # moving‐average return
    combined[f'avg_ret_{w}m'] = (
        combined.groupby('gvkey_x')['monthly_return']
                .rolling(window=w)
                .mean()
                .reset_index(level=0, drop=True)
    )
    # rolling volatility
    combined[f'vol_{w}m'] = (
        combined.groupby('gvkey_x')['monthly_return']
                .rolling(window=w)
                .std()
                .reset_index(level=0, drop=True)
    )

# fill initial NaNs
combined[[c for c in combined if c.startswith('avg_ret_')]] = \
    combined[[c for c in combined if c.startswith('avg_ret_')]].fillna(0)
combined[[c for c in combined if c.startswith('vol_')]] = \
    combined[[c for c in combined if c.startswith('vol_')]].fillna(0)

In [40]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24634 entries, 0 to 24633
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   ticker                              24634 non-null  object        
 1   tech_date                           24634 non-null  datetime64[ns]
 2   gvkey_x                             24634 non-null  int64         
 3   month_trading_volume                24634 non-null  float64       
 4   price_close_monthly                 24634 non-null  float64       
 5   monthly_return                      24634 non-null  float64       
 6   calendar_month                      24634 non-null  Int8          
 7   shares_outstanding_monthly          24634 non-null  float64       
 8   calendar_year                       24634 non-null  Int16         
 9   gics_industry                       24634 non-null  int64         
 10  gics_sector_x         

#### Defining the rules for scoring securities

### Technical Model

In [41]:
# add a 'stdev' column: the overall std dev of monthly_return per ticker
combined['stdev'] = combined.groupby('ticker')['monthly_return'].transform('std')

num_cols_tech = ['monthly_return','month_trading_volume','stdev', 'avg_ret_6m','avg_ret_12m','vol_6m', 'vol_12m']

In [42]:
# 2) Compute Sharpe Ratio at the ticker level and merge it back

# 3) Define your final feature sets
cat_cols = ['gics_sector_x']   # now only this in your categoricals
target   = 'sharpe_ratio'      # continuous target

# 4) split on date

train_end = '2020-12-31'

train = combined[combined['tech_date'] <= train_end]
test  = combined[combined['tech_date'] > train_end]

X_train = train[num_cols_tech + cat_cols]
X_test  = test [num_cols_tech + cat_cols]

y_train = train[target]
y_test  = test [target]

#### Pre processing and Pipeline

In [43]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# 1) Define preprocessing
pre = ColumnTransformer([
    ('num', StandardScaler(), num_cols_tech),
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['gics_sector_x'])
], remainder='drop')

# 2) Build the RF pipeline
rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=6,
    min_samples_leaf=5,
    random_state=2023,
    n_jobs=-1
)

rf_pipe = Pipeline([
    ('pre', pre),
    ('rf',  rf)
])

#### Fit and Evaluate

In [44]:
import warnings
warnings.filterwarnings('ignore')

# 3) Fit on the train set
rf_pipe.fit(X_train, y_train)

# 4) Predict on test
yt_pred = rf_pipe.predict(X_test)

# 5) Evaluate
print(" TEST  R²:", r2_score(y_test,  yt_pred))
print(" TEST RMSE:", np.sqrt(mean_squared_error(y_test,  yt_pred)))

 TEST  R²: 0.4154681679805
 TEST RMSE: 0.22795454440656904


#### Technical Model - Hyperparams


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

# 1) Parameter distributions for the Random Forest within the pipeline
param_dist = {
    # prepend 'rf__' because in rf_pipe the regressor step is named 'rf'
    'rf__n_estimators':    [100, 200, 500, 1000],
    'rf__max_depth':       [None, 5, 10, 15, 20],
    'rf__min_samples_leaf':[1, 2, 5, 10],
    'rf__max_features':    ['auto', 'sqrt', 'log2']
}

# 1) Split 'combined' into train (all data through 2020-12-31) and test (after)
train_tech = combined[combined['tech_date'] <= '2020-12-31']
test_tech  = combined[combined['tech_date'] >  '2020-12-31']

# 2) Define your tech feature lists
X_train = train_tech[num_cols_tech + cat_cols]
y_train = train_tech['sharpe_ratio']

X_test  = test_tech [num_cols_tech + cat_cols]
y_test  = test_tech ['sharpe_ratio']

rand_search = RandomizedSearchCV(
    estimator=rf_pipe,
    param_distributions=param_dist,
    n_iter=30,
    cv=TimeSeriesSplit(n_splits=5),
    scoring='r2',
    n_jobs=-1,
    verbose=2,
    random_state=42
)
rand_search.fit(X_train, y_train)   # now your tech X_train has the right cols

# 5) Inspect best parameters and CV score
print("Best parameters:", rand_search.best_params_)
print("Best CV R²:    ", rand_search.best_score_)

# 6) Re-evaluate on VALID and TEST
best_rf = rand_search.best_estimator_

y_test_pred  = best_rf.predict(X_test)

print(" TEST  R²:", r2_score(y_test,  y_test_pred))

Fitting 5 folds for each of 30 candidates, totalling 150 fits


### Fundamental Model

#### Feature Engineering

F-score calculation

In [ ]:
# 1) Compute “year-ago” baselines via a 12-month shift
grp = combined.groupby('ticker')

combined['roa_prev']          = grp['roa']                           .shift(12)
combined['oancfy_prev']       = grp['net_cash_flow_operating_activities'].shift(12)
combined['lev_ratio']         = safe_div(combined['long_term_debt'], combined['total_assets'])
combined['lev_prev']          = grp['lev_ratio']                     .shift(12)
combined['cr_prev']           = grp['current_ratio']                 .shift(12)
combined['shares_prev']       = grp['shares_outstanding_monthly']     .shift(12)
combined['profit_margin_prev']= grp['profit_margin']                 .shift(12)
combined['asset_turnover']    = safe_div(combined['total_revenue'],    combined['total_assets'])
combined['at_prev']           = grp['asset_turnover']                .shift(12)

# 2) Build each of the 9 Piotroski signals
s1 = (combined['roa']    >  0)                         .astype(int)  # positive ROA
s2 = (combined['net_cash_flow_operating_activities'] >  0)           .astype(int)  # positive CFO
s3 = (combined['roa']    >  combined['roa_prev'])    .astype(int)  # ROA ↑
s4 = (combined['net_cash_flow_operating_activities'] > combined['net_income']).astype(int)  # CFO > NI
s5 = (combined['lev_ratio'] < combined['lev_prev'])  .astype(int)  # Leverage ↓
s6 = (combined['current_ratio'] > combined['cr_prev']).astype(int)  # Liquidity ↑
s7 = (combined['shares_outstanding_monthly'] <= combined['shares_prev']).astype(int)  # No new equity
s8 = (combined['profit_margin'] > combined['profit_margin_prev']).astype(int)  # Margin ↑ (proxy)
s9 = (combined['asset_turnover'] > combined['at_prev']).astype(int)  # Asset Turnover ↑

# 3) Sum into a 0–9 F-Score
combined['f_score'] = s1 + s2 + s3 + s4 + s5 + s6 + s7 + s8 + s9

# 4) Clean up helper columns
combined = combined.drop(columns=[
    'roa_prev','oancfy_prev','lev_ratio','lev_prev',
    'cr_prev','shares_prev','profit_margin_prev',
    'asset_turnover','at_prev'
])

# 5) Quick sanity check
print("F-Score distribution:\n", combined['f_score'].value_counts().sort_index())

#### Preprocess, Fit, Evaluate and Tune

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import r2_score

# 1) Define fundamental feature list (no technicals)
fund_num_cols = [
    'current_assets','total_assets','common_equity_total',
    'current_debt','long_term_debt','depreciation_amortization',
    'preferred_dividends','current_liabilities','total_liabilities',
    'net_income','pretax_income','total_revenue',
    'total_income_taxes','interest_expense_total',
    'capital_expenditures','net_cash_flow_operating_activities',
    'dividends_per_share_quarter','price_low_quarter'
]
fund_cat_cols = ['gics_sector_x']

# 2) Re‐split into train (<= 2020-12-31) and test (> 2020-12-31)
train = combined[combined['tech_date'] <= '2020-12-31']
test  = combined[combined['tech_date'] >  '2020-12-31']

X_train = train[fund_num_cols + fund_cat_cols]
y_train = train['f_score']        # Piotroski target
X_test  = test [fund_num_cols + fund_cat_cols]
y_test  = test ['f_score']

# 3) Preprocessing & RF pipeline
pre_fund = ColumnTransformer([
    ('num', StandardScaler(),      fund_num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), fund_cat_cols),
], remainder='drop')

rf_fund_pipe = Pipeline([
    ('prep', pre_fund),
    ('rf',   RandomForestRegressor(random_state=42, n_jobs=-1))
])

# 4) Hyperparameter search over the training fold
param_dist = {
    'rf__n_estimators':    [100, 200, 500, 1000],
    'rf__max_depth':       [None, 5, 10, 20],
    'rf__min_samples_leaf':[1, 2, 5, 10],
    'rf__max_features':    ['auto', 'sqrt', 'log2']
}

tscv = TimeSeriesSplit(n_splits=5)

search_fund = RandomizedSearchCV(
    rf_fund_pipe,
    param_distributions=param_dist,
    n_iter=30,
    cv=tscv,
    scoring='r2',
    n_jobs=-1,
    verbose=2,
    random_state=42
)
search_fund.fit(X_train, y_train)

print("Best Params:", search_fund.best_params_)
print("Best CV R²: ", search_fund.best_score_)

# 5) Final evaluation on the hold-out test set
best_fund_model = search_fund.best_estimator_
y_pred_test     = best_fund_model.predict(X_test)

print("TEST R²:", r2_score(y_test, y_pred_test))


In [ ]:
# tech model prediction
combined['sharpe_ratio_pred'] = best_rf.predict(
    combined[num_cols_tech + cat_cols]
)

# fundamental model prediction
combined['f_score_pred'] = best_fund_model.predict(
    combined[fund_num_cols + fund_cat_cols]
)

### Technical and Fundamental Scores  

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(1,10))

# .to_numpy() or .values ensures we get a proper (n_rows, 2) array
pred_matrix = combined[['sharpe_ratio_pred','f_score_pred']].to_numpy()

# Scale into [1, 10]
tech_fund_scores = scaler.fit_transform(pred_matrix)

# Assign back to two new columns
combined['tech_score_10'] = tech_fund_scores[:, 0]
combined['fund_score_10'] = tech_fund_scores[:, 1]

In [ ]:
combined.head()

In [ ]:
combined.groupby('ticker')['tech_score_10'].mean().sort_values(ascending=False)

In [ ]:
combined.groupby('ticker')['fund_score_10'].mean().sort_values(ascending=False)

END.